In [ ]:
import numpy as np
import tensorflow as tf
import os

# load data

In [ ]:
train, test_val = tf.keras.datasets.mnist.load_data()

# Data preprocessing

In [ ]:
# scale data to obtain data between [0, 1]
user_test, train, test_val = (train[0][:10] / 255.0, train[1][:10]), \
                           (train[0][10:] / 255.0, train[1][10:]), \
                           (test_val[0] / 255.0, test_val[1])

In [ ]:
# split test_val into validation, and test
VAL2TEST_RATIO = 0.6
NUM_VAL = int(test_val[0].shape[0] * VAL2TEST_RATIO)
validation = (test_val[0][:NUM_VAL,:,:], test_val[1][:NUM_VAL,])
test = (test_val[0][NUM_VAL:,:,:], test_val[1][NUM_VAL:,])

In [ ]:
# shuffle and batch data
BUFFER_SIZE = 10000
BATCH_SIZE = 50

train_ds = tf.data.Dataset.from_tensor_slices(train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_ds = tf.data.Dataset.from_tensor_slices(validation).batch(validation[0].shape[0])
test_ds = tf.data.Dataset.from_tensor_slices(test).batch(test[0].shape[0])
user_ds = tf.data.Dataset.from_tensor_slices(user_test).batch(user_test[0].shape[0])

# Build model

In [ ]:
input_shape = (28, 28, 1)
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape, name="flatten"),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax', name='last_layer')
])

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(optimizer='adam', loss=loss_object, metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 10

model.fit(train_ds, epochs=NUM_EPOCHS, validation_data=validation_ds, verbose=2)

In [ ]:
save_path = os.path.join("model_2")
tf.saved_model.save(model, save_path)

# model evaluate

In [ ]:
my_model = tf.saved_model.load("./model_2")

In [ ]:
prediction = my_model.signatures["serving_default"]

In [ ]:
# expected: TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='flatten_input')
user_input = tf.convert_to_tensor(user_test[0][0], np.float32)
prediction(flatten_input=user_input), user_test[1][0]